In [389]:
import pandas as pd
import datetime
df = pd.read_csv( "castle-event-data-v4.csv",sep=',', header=0, parse_dates=True,quoting=1 )


In [390]:
import numpy as np

#### replace all values "custome" in column name with custom name

In [391]:
df.name=df.name.replace(to_replace='custom',value=np.nan)
df.name=df.name.fillna(df.custom_name)

#### read in the agent os file i generated from user agent API

In [392]:
def agent_name_from_api(ua_unique,file_name='agent_name.csv'):
    i=0    
    if not os.path.isfile(file_name):
        for ua in ua_unique:
            if i%500==0:
                print('finishing {0} out of {1} agent'.format(i,len(ua_unique)))
            user_agen_api = requests.get("http://www.useragentstring.com/?uas="+ua+'&getJSON=os_name-agent_name')
            if user_agen_api.status_code==200:
                user_agent[ua]=[user_agen_api.json()['os_name'],user_agen_api.json()['agent_name']]
            i+=1
        df_ua=pd.DataFrame.from_dict(user_agent,orient='index',)
#        df_ua.column=['user_agent']
        df_ua.to_csv('agent_name.csv')
    else: df_ua=pd.read_csv('agent_name.csv')
    return df_ua

In [393]:
#read in the agent os file i generated from user agent API
df_agent_os=pd.read_csv('agent_name.csv')

In [394]:
df_agent_os.columns.values

array(['Unnamed: 0', '0', '1'], dtype=object)

In [395]:
df_agent_os.rename(columns={'Unnamed: 0':'ua','0':'os','1':'agent_name'},inplace=True)

In [396]:
df_agent_os.head()

,ua,os,agent_name
0,Mozilla/5.0 (Linux; Android 5.1.1; Ilium_PAD_i...,Android,Android Webkit Browser
1,Mozilla/5.0 (Linux; Android 4.4.2; SM-G355M Bu...,Android,Android Webkit Browser
2,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,Windows NT,Internet Explorer
3,Mozilla/5.0 (Linux; Android 5.0.1; SM-N910P Bu...,Android,Android Webkit Browser
4,Mozilla/5.0 (Linux; Android 4.4.4; ILIUM L900 ...,Android,Android Webkit Browser


In [397]:
df=df.merge(df_agent_os,left_on='ua',right_on='ua',how='left')

####  Calculate the percentile of each time difference between actions

In [398]:
df.sort_values(by=['device_id','created_at'],ascending=True,inplace=True)
device_grouped=df.groupby(by='device_id')
df['time_diff']=device_grouped.created_at.diff()

In [399]:
df.time_diff=df.time_diff.fillna(df.time_diff.mean())

In [400]:
df['time_diff_pct']=df.time_diff.rank(pct=True)

In [401]:
df.columns

Index([u'created_at', u'user_id', u'session_id', u'device_id', u'ua', u'l',
       u'name', u'custom_name', u'ip', u'continent', u'country', u'city',
       u'lon', u'lat', u'accuracy', u'isp_name', u'datacenter', u'proxy',
       u'mobile_device', u'os', u'agent_name', u'time_diff', u'time_diff_pct'],
      dtype='object')

#### Calculate the speed ofuser travels

In [402]:
df['lon_diff']=device_grouped.lon.diff()
df['lat_diff']=device_grouped.lat.diff()

In [403]:
df['speed']=(df.lon_diff.pow(2)+df.lat_diff.pow(2))/df.time_diff

In [404]:
df.speed=df.speed.fillna(0)

#### Calculate the percentile of each time difference between actions, country, isp_name, agent_name

In [405]:
# function to convert categorical to percentile
def cat_to_pct(df,col):
    pct=df[col].value_counts(dropna=False)/df[col].notnull().sum()
    df=df.join(pct,on=[col],how='left',rsuffix='_pct')
    return df

In [406]:
df=cat_to_pct(df,'os')

In [407]:
df=cat_to_pct(df,'country')

In [408]:
df=cat_to_pct(df,'continent')

In [409]:
df=cat_to_pct(df,'isp_name')

In [410]:
df=cat_to_pct(df,'agent_name')

In [411]:
df=cat_to_pct(df,'name')

In [412]:
df.columns

Index([u'created_at', u'user_id', u'session_id', u'device_id', u'ua', u'l',
       u'name', u'custom_name', u'ip', u'continent', u'country', u'city',
       u'lon', u'lat', u'accuracy', u'isp_name', u'datacenter', u'proxy',
       u'mobile_device', u'os', u'agent_name', u'time_diff', u'time_diff_pct',
       u'lon_diff', u'lat_diff', u'speed', u'os_pct', u'country_pct',
       u'continent_pct', u'isp_name_pct', u'agent_name_pct', u'name_pct'],
      dtype='object')

In [413]:
device_id=df.device_id.copy()
df_model=df.drop(['created_at','user_id','session_id','device_id','ua','l','name','custom_name'\
                  ,'ip','continent','country','city','lon','lat','accuracy','isp_name','os','agent_name'\
                  ,'lon_diff','lat_diff'\
                 ,'time_diff'],axis=1)
#df_model=df_model.drop(['l','city','isp_name'],axis=1)

#### Normalize df_model to input into the model

In [414]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

In [415]:
df_model.name_pct=min_max_scaler.fit_transform(df_model.name_pct.reshape(-1,1))

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':


In [416]:
df_model.datacenter=df_model.datacenter.astype('int')*0.5
df_model.proxy=df_model.proxy.astype('int')*0.5
df_model.mobile_device=df_model.mobile_device.astype('int')*0.5

In [417]:
#initialize isolation forest
from sklearn.ensemble import IsolationForest
seed=25
sf=IsolationForest(contamination=0.001,n_jobs=-1,random_state=seed)

In [ ]:
# fitting the isolation forest model
sf.fit(df_model)


IsolationForest(bootstrap=False, contamination=0.001, max_features=1.0,
        max_samples='auto', n_estimators=100, n_jobs=-1, random_state=25,
        verbose=0)

In [ ]:
predict_anom=sf.predict(df_model)

In [ ]:
df_model['score']=predict_anom

In [ ]:
df['score']=predict_anom

In [ ]:
df_anom=df[df.score==-1]

In [ ]:
df_anom.columns

In [ ]:
df_anom.iloc[:,:20].head()

In [ ]:
# columns in model: 'name', u'continent', u'country', u'datacenter', u'proxy',
#       u'mobile_device'
df_anom.name.value_counts(dropna=False)

In [ ]:
df.name.value_counts()

In [ ]:
anom_gp=df_anom.groupby(by='device_id').created_at.count()

In [ ]:
result=anom_gp.sort_values(ascending=False)
result

In [ ]:
df[(df.device_id==8066019)&(df.score==-1)].iloc[:,-20:]